In [20]:
library(limma)
library(GEOquery)
library(WGCNA)
library(dplyr)

In [21]:
gse <- getGEO("GSE106582", GSEMatrix = TRUE)

Found 1 file(s)

GSE106582_series_matrix.txt.gz

Using locally cached version: /tmp/RtmpJsLRW9/GSE106582_series_matrix.txt.gz

Using locally cached version of GPL10558 found here:
/tmp/RtmpJsLRW9/GPL10558.soft.gz 



In [22]:
expr <- exprs(gse[[1]])
pheno <- pData(gse[[1]])
fdata <- fData(gse[[1]])

In [23]:
write.csv(fdata, "fdata.csv", row.names = TRUE)

In [24]:
head(expr)

,GSM2843099,GSM2843100,GSM2843101,GSM2843102,GSM2843103,GSM2843104,GSM2843105,GSM2843106,GSM2843107,GSM2843108,⋯,GSM2843283,GSM2843284,GSM2843285,GSM2843286,GSM2843287,GSM2843288,GSM2843289,GSM2843290,GSM2843291,GSM2843292
ILMN_1343291,14.246636,14.306430,14.348227,14.233364,14.037613,14.306430,14.209482,14.233364,14.137572,14.348227,⋯,14.075304,14.209482,14.262302,14.137572,14.185752,14.306430,14.193309,14.348227,14.280982,14.246636
ILMN_1343295,12.738228,12.206211,12.700099,12.029697,11.334419,12.816596,12.552413,12.822330,11.944893,12.144393,⋯,11.679008,11.843007,12.866065,11.624542,11.237864,11.465310,11.443065,10.883836,11.750486,11.466828
ILMN_1651199,6.412016,6.633564,6.574890,6.450712,6.579476,6.439554,6.431542,6.346484,6.518873,6.325833,⋯,6.420452,6.674008,6.711727,6.579345,6.388410,6.399378,6.889240,6.394718,6.516751,6.187478
ILMN_1651209,6.710966,6.805210,6.622005,6.668240,6.736337,6.703673,6.452449,6.592954,6.502550,6.656889,⋯,6.612033,6.555067,6.474639,6.650995,6.624086,6.799771,6.573304,6.542434,6.601863,6.723900
ILMN_1651210,6.512163,6.588000,6.532367,6.576099,6.343878,6.589865,6.501190,6.571158,6.659456,6.476374,⋯,6.470704,6.419202,6.517013,6.552662,6.433940,6.433472,6.387217,6.443103,6.389635,6.528619
ILMN_1651221,6.594800,6.556280,6.739953,6.621620,6.474938,6.607903,6.558625,6.456971,6.564148,6.600523,⋯,6.696744,6.628282,6.537876,6.755657,6.498592,6.568854,6.553839,6.688859,6.424253,6.568933


In [25]:
head(pheno)

,title,geo_accession,status,submission_date,last_update_date,type,channel_count,source_name_ch1,organism_ch1,characteristics_ch1,⋯,contact_city,contact_zip/postal_code,contact_country,supplementary_file,data_row_count,age:ch1,disease state:ch1,gender:ch1,patientid:ch1,tissue:ch1
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
GSM2843099,400072_A,GSM2843099,Public on Dec 01 2018,Nov 06 2017,Dec 01 2018,RNA,1,Patient 400072; Tissue tumor,Homo sapiens,patientid: 400072,⋯,Freiburg,79110,Germany,NONE,47290,76,Colorectal cancer patient,M,400072,tumor
GSM2843100,400072_C,GSM2843100,Public on Dec 01 2018,Nov 06 2017,Dec 01 2018,RNA,1,Patient 400072; Tissue mucosa,Homo sapiens,patientid: 400072,⋯,Freiburg,79110,Germany,NONE,47290,76,Colorectal cancer patient,M,400072,mucosa
GSM2843101,400152_A,GSM2843101,Public on Dec 01 2018,Nov 06 2017,Dec 01 2018,RNA,1,Patient 400152; Tissue tumor,Homo sapiens,patientid: 400152,⋯,Freiburg,79110,Germany,NONE,47290,34,Colorectal cancer patient,M,400152,tumor
GSM2843102,400152_C,GSM2843102,Public on Dec 01 2018,Nov 06 2017,Dec 01 2018,RNA,1,Patient 400152; Tissue mucosa,Homo sapiens,patientid: 400152,⋯,Freiburg,79110,Germany,NONE,47290,34,Colorectal cancer patient,M,400152,mucosa
GSM2843103,400200_C,GSM2843103,Public on Dec 01 2018,Nov 06 2017,Dec 01 2018,RNA,1,Patient 400200; Tissue mucosa,Homo sapiens,patientid: 400200,⋯,Freiburg,79110,Germany,NONE,47290,53,Colorectal cancer patient,M,400200,mucosa
GSM2843104,400216_A,GSM2843104,Public on Dec 01 2018,Nov 06 2017,Dec 01 2018,RNA,1,Patient 400216; Tissue tumor,Homo sapiens,patientid: 400216,⋯,Freiburg,79110,Germany,NONE,47290,56,Colorectal cancer patient,M,400216,tumor


In [26]:
head(fdata, 1)

,ID,Species,Source,Search_Key,Transcript,ILMN_Gene,Source_Reference_ID,RefSeq_ID,Unigene_ID,Entrez_Gene_ID,⋯,Probe_Chr_Orientation,Probe_Coordinates,Cytoband,Definition,Ontology_Component,Ontology_Process,Ontology_Function,Synonyms,Obsolete_Probe_Id,GB_ACC
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
ILMN_1343291,ILMN_1343291,Homo sapiens,RefSeq,NM_001402.4,ILMN_5311,EEF1A1,NM_001402.5,NM_001402.5,,1915,⋯,-,74284362-74284378:74284474-74284506,6q13c,"Homo sapiens eukaryotic translation elongation factor 1 alpha 1 (EEF1A1), mRNA.","All of the contents of a cell excluding the plasma membrane and nucleus, but including other subcellular structures [goid 5737] [evidence IEA]; All of the contents of a cell excluding the plasma membrane and nucleus, but including other subcellular structures [goid 5737] [evidence IEA]; All of the contents of a cell excluding the plasma membrane and nucleus, but including other subcellular structures [goid 5737] [pmid 3512269] [evidence TAS]; That part of the cytoplasm that does not contain membranous or particulate subcellular components [goid 5829] [pmid 10368288] [evidence EXP]; A multisubunit nucleotide exchange complex that binds GTP and aminoacyl-tRNAs, and catalyzes their codon-dependent placement at the A-site of the ribosome. In humans, the complex is composed of four subunits, alpha, beta, delta and gamma [goid 5853] [pmid 2564392] [evidence TAS]",The successive addition of amino acid residues to a nascent polypeptide chain during protein biosynthesis [goid 6414] [evidence IEA]; The successive addition of amino acid residues to a nascent polypeptide chain during protein biosynthesis [goid 6414] [pmid 8812466] [evidence TAS]; The successive addition of amino acid residues to a nascent polypeptide chain during protein biosynthesis [goid 6414] [pmid 15189156] [evidence EXP],"Interacting selectively with a nucleotide, any compound consisting of a nucleoside that is esterified with (ortho)phosphate or an oligophosphate at any hydroxyl group on the ribose or deoxyribose moiety [goid 166] [evidence IEA]; Functions in chain elongation during polypeptide synthesis at the ribosome [goid 3746] [evidence IEA]; Catalysis of the reaction: GTP + H2O = GDP + phosphate [goid 3924] [evidence IEA]; Interacting selectively with GTP, guanosine triphosphate [goid 5525] [evidence IEA]; Interacting selectively with a nucleotide, any compound consisting of a nucleoside that is esterified with (ortho)phosphate or an oligophosphate at any hydroxyl group on the ribose or deoxyribose moiety [goid 166] [evidence IEA]; Functions in chain elongation during polypeptide synthesis at the ribosome [goid 3746] [evidence IEA]; Catalysis of the reaction: GTP + H2O = GDP + phosphate [goid 3924] [evidence IEA]; Interacting selectively with GTP, guanosine triphosphate [goid 5525] [evidence IEA]; Interacting selectively with a nucleotide, any compound consisting of a nucleoside that is esterified with (ortho)phosphate or an oligophosphate at any hydroxyl group on the ribose or deoxyribose moiety [goid 166] [evidence IEA]; Catalysis of the reaction: GTP + H2O = GDP + phosphate [goid 3924] [evidence IEA]; Interacting selectively with GTP, guanosine triphosphate [goid 5525] [evidence IEA]; Interacting selectively with a nucleotide, any compound consisting of a nucleoside that is esterified with (ortho)phosphate or an oligophosphate at any hydroxyl group on the ribose or deoxyribose moiety [goid 166] [evidence IEA]; Catalysis of the reaction: GTP + H2O = GDP + phosphate [goid 3924] [evidence IEA]; Interacting selectively with GTP, guanosine triphosphate [goid 5525] [evidence IEA]; Interacting selectively with a nucleotide, any compound consisting of a nucleoside that is esterified with (ortho)phosphate or an oligophosphate at any hydroxyl group on the ribose or deoxyribose moiety [goid 166] [evidence IEA]; Functions in chain elongation during polypeptide synthe

In [27]:
colnames(fdata)

[1] "ID"                    "Species"               "Source"               
 [4] "Search_Key"            "Transcript"            "ILMN_Gene"            
 [7] "Source_Reference_ID"   "RefSeq_ID"             "Unigene_ID"           
[10] "Entrez_Gene_ID"        "GI"                    "Accession"            
[13] "Symbol"                "Protein_Product"       "Probe_Id"             
[16] "Array_Address_Id"      "Probe_Type"            "Probe_Start"          
[19] "SEQUENCE"              "Chromosome"            "Probe_Chr_Orientation"
[22] "Probe_Coordinates"     "Cytoband"              "Definition"           
[25] "Ontology_Component"    "Ontology_Process"      "Ontology_Function"    
[28] "Synonyms"              "Obsolete_Probe_Id"     "GB_ACC"

In [28]:
probe2gene_map <- fdata[, c("ID", "Symbol")]
colnames(probe2gene_map) <- c("ProbeID", "GeneSymbol")

# Remove any rows where the GeneSymbol is missing (NA) or empty
probe2gene_map <- probe2gene_map[!is.na(probe2gene_map$GeneSymbol) & probe2gene_map$GeneSymbol != "", ]
head(probe2gene_map)

,ProbeID,GeneSymbol
,<chr>,<chr>
ILMN_1343291,ILMN_1343291,EEF1A1
ILMN_1343295,ILMN_1343295,GAPDH
ILMN_1651199,ILMN_1651199,LOC643334
ILMN_1651209,ILMN_1651209,SLC35E2
ILMN_1651210,ILMN_1651210,DUSP22
ILMN_1651221,ILMN_1651221,LOC642820


In [29]:
# 1. Get the probe IDs that are actually present in your expression matrix
# Your expression matrix rownames

expression_matrix <- expr

expr_probes <- rownames(expression_matrix)

# 2. Filter the mapping table to only include probes present in the expression matrix
valid_map <- probe2gene_map[probe2gene_map$ProbeID %in% expr_probes, ]

# 3. Calculate the average expression for each probe in the matrix
probe_means <- rowMeans(expression_matrix, na.rm = TRUE)

# 4. Add this average expression to the mapping dataframe
# (Match by ProbeID)
valid_map$MeanExpression <- probe_means[valid_map$ProbeID]

# 5. For each gene, find the probe with the highest average expression

probes_to_keep <- valid_map %>%
  group_by(GeneSymbol) %>%
  arrange(desc(MeanExpression)) %>% # Sort by expression, highest first
  dplyr::slice(1) %>%               # Take the first row (highest expr) for each gene
  ungroup()

# 6. Filter your original expression matrix to only include these "best representative" probes
expression_matrix_gene_level <- expression_matrix[probes_to_keep$ProbeID, ]

# 7. Finally, replace the rownames (probe IDs) with the Gene Symbols
rownames(expression_matrix_gene_level) <- probes_to_keep$GeneSymbol

# Optional: Remove the temporary 'MeanExpression' column from the map if you want to keep it clean
probes_to_keep$MeanExpression <- NULL

In [30]:
head(expression_matrix_gene_level, 20)

,GSM2843099,GSM2843100,GSM2843101,GSM2843102,GSM2843103,GSM2843104,GSM2843105,GSM2843106,GSM2843107,GSM2843108,⋯,GSM2843283,GSM2843284,GSM2843285,GSM2843286,GSM2843287,GSM2843288,GSM2843289,GSM2843290,GSM2843291,GSM2843292
1-Dec,6.346731,6.497192,6.375568,6.442496,6.708249,6.550256,6.406398,6.647431,6.476604,6.459665,⋯,6.346629,6.467283,6.422234,6.515479,6.702639,6.586317,6.511672,6.597557,6.564734,6.542214
1-Mar,7.049863,6.646066,6.831677,6.571903,6.787348,6.391930,6.770967,7.104668,6.718457,6.861702,⋯,6.852220,6.857612,6.781521,6.969206,6.990639,6.849225,7.111658,7.479136,7.441931,6.888703
10-Mar,6.388496,6.479169,6.476078,6.502055,6.717118,6.586716,6.586192,6.685143,6.430921,6.639820,⋯,6.392254,6.365890,6.369265,6.522212,6.436309,6.716925,6.440618,6.422365,6.475643,6.563881
11-Mar,6.337497,6.511963,6.467740,6.506580,6.487988,6.557287,6.360386,6.506761,6.616031,6.477945,⋯,6.339700,6.641328,6.588288,6.535084,6.455771,6.548076,6.581844,6.555996,6.449674,6.452908
2-Mar,8.873265,9.244716,7.903894,9.650984,8.052109,7.654220,8.092806,8.346829,9.923108,8.356884,⋯,8.808454,7.953702,7.140647,7.464895,7.435755,7.891693,7.746457,8.864715,7.991454,7.900150
3-Mar,7.811793,8.488025,8.790167,8.094432,9.294317,9.388382,8.864715,8.402515,7.774851,8.488796,⋯,8.440810,9.386033,8.185411,9.205678,9.479951,8.767132,8.721996,8.105148,8.777152,9.404472
4-Mar,6.531090,6.479849,6.419195,6.529403,6.588301,6.469661,6.646628,6.443487,6.516350,6.553150,⋯,6.436135,6.550838,6.334187,6.413506,6.678824,6.382063,6.433924,6.365127,6.519616,6.676666
5-Mar,7.756915,7.818265,7.961034,7.897534,8.332983,7.847909,7.881862,7.888485,8.022935,7.811584,⋯,8.079966,7.925815,7.802880,7.972215,7.886475,8.125270,7.554336,7.874202,7.874831,8.024762
6-Mar,9.506719,10.646609,10.411151,10.179333,10.243798,9.357304,9.784679,9.586027,9.892513,9.994921,⋯,10.750454,9.998761,9.494629,10.078739,10.193928,10.951498,9.689115,10.106460,9.251079,9.716908
7-Mar,10.342044,10.457642,10.725027,10.710924,10.073583,9.642545,9.788929,10.040531,10.833201,10.320163,⋯,10.425449,10.049160,10.099394,10.182909,10.360297,9.894959,10.152783,10.704827,10.306369,10.378865


In [31]:
expr <- expression_matrix_gene_level

In [32]:
table(pheno$'tissue:ch1')


mucosa  tumor 
   117     77 

In [33]:
group <- factor(pheno$'tissue:ch1')

In [34]:
# differential expression analysis with limma
design <- model.matrix(~0 + group)
colnames(design) <- levels(group)

fit <- lmFit(expr, design)
contrast.matrix <- makeContrasts(TumorVsMucosa = tumor - mucosa, levels=design)
fit2 <- contrasts.fit(fit, contrast.matrix)
fit2 <- eBayes(fit2)

In [35]:
# Results table
deg_results <- topTable(fit2, coef="TumorVsMucosa", number=Inf, adjust="fdr")

# Add probe annotation
deg_results <- cbind(fdata[rownames(deg_results), ], deg_results)

# Apply thresholds (like the paper)
deg_filtered <- subset(deg_results, adj.P.Val < 0.05 & abs(logFC) >= 0.5)

In [36]:
# Quick summary
cat("Total DEGs (with symbols):", nrow(deg_filtered), "\n")

Total DEGs (with symbols): 2694 


In [37]:
# Save all results
write.csv(deg_results,
          file = "GSE106582_DEG_results_all.csv",
          row.names = TRUE)

# Save filtered DEGs
write.csv(deg_filtered,
          file = "GSE106582_DEG_results_filtered.csv",
          row.names = TRUE)